In [ ]:
import pandas as pd
import PyPDF2
import re

In [ ]:
# Definindo os caminhos dos arquivos
csv_paths = {
    'bradesco': r"SEU PATH",
    'cora': r"SEU PATH",
    'pagseguro': r"SEU PATH"
}
pdf_path = r"SEU PATH"

In [ ]:
# Função para carregar dados CSV
def load_csv_data(path, delimiter=',', skiprows=None, encoding=None):
    return pd.read_csv(path, delimiter=delimiter, skiprows=skiprows, encoding=encoding)


In [ ]:
# Carregar dados do CSV da Cora
df_cora = load_csv_data(csv_paths['cora'])

# Renomear os campos
df_cora.rename(columns={
    'Data': 'data',
    'Tipo Transação': 'tipo_transacao',
    'Transação': 'categoria_transacao',
    'Identificação': 'desc_transação',
    'Valor': 'valor'
}, inplace=True)

# Incluir a coluna codigo_transacao como o número da linha no DataFrame
df_cora['codigo_transacao'] = df_cora.index + 1

# Incluir nome do banco
df_cora['banco'] = "Cora"

# Reorganizar as colunas na ordem desejada
df_cora = df_cora[['data', 'codigo_transacao', 'tipo_transacao', 'categoria_transacao', 'desc_transação', 'valor','banco']]




In [ ]:
# Carregar e processar dados do CSV do PagSeguro
df_pagbk = load_csv_data(csv_paths['pagseguro'], delimiter=';')


# Renomear os campos
df_pagbk.rename(columns={
    'DATA': 'data',
    'TIPO': 'categoria_transacao',
    'CODIGO DA TRANSACAO': 'codigo_transacao',
    'DESCRICAO': 'desc_transação',
    'VALOR': 'valor'
}, inplace=True)

# Ajustar o formato do campo 'valor'
df_pagbk['valor'] = df_pagbk['valor'].astype(str).str.replace(',', '.').astype(float)

# Criar a coluna 'tipo' que indica Débito ou Crédito com base no valor da transação
df_pagbk['tipo_transacao'] = df_pagbk['valor'].apply(lambda x: 'Crédito' if x >= 0 else 'Débito')

# Incluir nome do banco
df_pagbk['banco'] = "Pagbank"

# Reorganizar as colunas na ordem desejada
df_pagbk = df_pagbk[['data', 'codigo_transacao', 'tipo_transacao', 'categoria_transacao', 'desc_transação', 'valor','banco']]


In [ ]:
# Carregar e processar dados do CSV do Bradesco
df_bra = load_csv_data(csv_paths['bradesco'], delimiter=';', skiprows=2, encoding='ISO-8859-1')
df_bra.columns = ['Data', 'Transação', 'Codigo da transação.', 'Credito', 'Debito', 'Saldo']

# Renomear os campos
df_bra.rename(columns={
    'Data': 'data',
    'Codigo da transação.': 'codigo_transacao',
    'Transação': 'desc_transação',
}, inplace=True)

# Criar uma coluna vazia chamada categoria_transacao
df_bra['categoria_transacao'] = ''

total_index = df_bra[df_bra['data'].str.contains('Total', na=False)].index
if not total_index.empty:
    df_bra = df_bra.iloc[:total_index[0]]
df_bra = df_bra.dropna(subset=['data'])
df_bra = df_bra[~df_bra['desc_transação'].str.contains('SALDO ANTERIOR', na=False)]
for col in ['Credito', 'Debito', 'Saldo']:
    df_bra[col] = df_bra[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Criar DataFrames separados para créditos e débitos
df_credito = df_bra[['data', 'desc_transação', 'codigo_transacao', 'Credito', 'categoria_transacao']].copy()
df_credito['tipo_transacao'] = 'Crédito'
df_credito.rename(columns={'Credito': 'valor'}, inplace=True)

df_debito = df_bra[['data', 'desc_transação', 'codigo_transacao', 'Debito', 'categoria_transacao']].copy()
df_debito['tipo_transacao'] = 'Débito'
df_debito.rename(columns={'Debito': 'valor'}, inplace=True)

# Concatenar os DataFrames de crédito e débito
df_final = pd.concat([df_credito, df_debito], ignore_index=True)

# Remover linhas com valores nulos na coluna 'valor'
df_bra = df_final.dropna(subset=['valor'])

# Verificar se a coluna 'categoria_transacao' ainda existe, caso contrário, recriá-la
if 'categoria_transacao' not in df_bra.columns:
    df_bra['categoria_transacao'] = ''

# Incluir nome do banco
df_bra['banco'] = "Bradesco"

# Reorganizar as colunas na ordem desejada
df_bra = df_bra[['data', 'codigo_transacao', 'tipo_transacao', 'categoria_transacao', 'desc_transação', 'valor','banco']]



In [ ]:
# Função para extrair texto de um PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''.join([page.extract_text() for page in reader.pages])
    return text

In [ ]:
# Função para processar texto extraído e criar um DataFrame
def parse_text_to_dataframe(text):
    pattern = re.compile(
        r'(\d{2}-\d{2}-\d{4})'  # Data
        r'([\s\S]*?)'  # Descrição
        r'(\d{11})'  # ID da operação
        r'\s+R\$\s+([\d.,-]+)'  # Valor
        r'\s+R\$\s+([\d.,]+)'  # Saldo
    )
    matches = pattern.findall(text)
    df = pd.DataFrame(matches, columns=['Data', 'Descrição', 'ID da operação', 'Valor', 'Saldo'])
    df['Descrição'] = df['Descrição'].str.strip()
    df['Valor'] = df['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)
    df['Saldo'] = df['Saldo'].str.replace('.', '').str.replace(',', '.').astype(float)
    return df

In [ ]:
text = extract_text_from_pdf(pdf_path)
df_mcpgo = parse_text_to_dataframe(text)

# Renomear os campos
df_mcpgo.rename(columns={
    'Data': 'data',
    'ID da operação': 'codigo_transacao',
    'Descrição': 'desc_transação',
    'Valor': 'valor',
}, inplace=True)

# Remover a coluna 'Saldo'
df_mcpgo.drop(columns=['Saldo'], inplace=True)

# Criar a coluna 'tipo' com base no valor da transação
df_mcpgo['tipo_transacao'] = df_mcpgo['valor'].apply(lambda x: 'Débito' if x < 0 else 'Crédito')
# Adicionar a coluna 'categoria_transacao' vazia
df_mcpgo['categoria_transacao'] = ''

# Incluir nome do banco
df_mcpgo['banco'] = "Mercado Pago"

# Reorganizar as colunas na ordem desejada
df_mcpgo = df_mcpgo[['data', 'codigo_transacao', 'tipo_transacao', 'categoria_transacao', 'desc_transação', 'valor','banco']]

# Mostrar as primeiras linhas do DataFrame
df_mcpgo.head()

In [ ]:
df_union = pd.concat([df_mcpgo, df_bra, df_pagbk, df_cora], ignore_index=True)

In [ ]:
df_union.head()